In [34]:
# import team_utils as tu
import pandas as pd
# import numpy as np
# import requests
import my_utils2 as mu
from tqdm import tqdm
import json
import datetime


In [35]:
def rawdataLoader(datas: str):
    return json.loads(datas.replace("'", '"').replace('True', '1').replace('False', '0'))

In [36]:
now = datetime.datetime.now()
print(now)

2023-05-11 11:32:29.622702


In [37]:
df = mu.oracle_totalExecute("SELECT version,game_id, champion_id,participant_number, champion, startitem FROM RAWDATA")
df = df.sort_values(['GAME_ID','PARTICIPANT_NUMBER'])
df_cols = df.columns[4:]

now = datetime.datetime.now()
now = now.strftime('%Y-%m-%d %H:%M:%S')

matches = {a:[] for a in df_cols}
for rec in tqdm(df.iloc):
    for col in [a for a in df_cols]:
        matches[col].append(rawdataLoader(rec[col]))
for col in [a for a in df_cols]:
      df[col] = matches[col]


version_lst = df['VERSION'].to_list()
champid_lst = df['CHAMPION_ID'].to_list()
champion_list =  df['CHAMPION'].tolist()
startitem_list =  df['STARTITEM'].tolist()
    
result = []
col = ['version','update_time','championId','lane','stratitem','win']
    
for idx, champ in enumerate(tqdm(champion_list)):
    result.append([
        version_lst[idx],
        now,
        champ['championId'],
        champ['teamPosition'],
        str(startitem_list[idx]),
        int(champ['win'])
        ])
champ_item_start = pd.DataFrame(result, columns=col)

oracle open!
oracle close!


29552it [00:04, 6290.29it/s]
100%|████████████████████████████████████████████████████████████████████████| 29552/29552 [00:00<00:00, 314570.14it/s]


In [38]:
champ_item_start

,version,update_time,championId,lane,stratitem,win
0,12.19.471.6581,2023-05-11 11:34:18,57,TOP,"[1054, 2003]",1
1,12.19.471.6581,2023-05-11 11:34:18,154,JUNGLE,"[1039, 2031, 3340]",1
2,12.19.471.6581,2023-05-11 11:34:18,34,MIDDLE,"[1056, 2003, 2003]",1
3,12.19.471.6581,2023-05-11 11:34:18,222,BOTTOM,"[1055, 2003]",1
4,12.19.471.6581,2023-05-11 11:34:18,32,UTILITY,"[3858, 2003, 2003]",1
...,...,...,...,...,...,...
29547,13.9.506.4846,2023-05-11 11:34:18,92,TOP,"[1055, 2003]",0
29548,13.9.506.4846,2023-05-11 11:34:18,154,JUNGLE,"[1103, 3340, 2003]",0
29549,13.9.506.4846,2023-05-11 11:34:18,157,MIDDLE,"[1055, 2003, 3340]",0
29550,13.9.506.4846,2023-05-11 11:34:18,222,BOTTOM,"[1055, 2003]",0


In [8]:
startitem_list =  df['STARTITEM'].tolist()

In [40]:
champ_item_start.iloc[0]

championId                          17
lane                               TOP
stratitem     [1056, 2003, 2003, 3340]
win                                  0
Name: 0, dtype: object

In [63]:
champ_item_start.dtypes

championId     int64
lane          object
stratitem     object
win            int64
dtype: object

In [64]:
champ_item_start

,championId,lane,stratitem,win
0,17,TOP,"[1056, 2003, 2003, 3340]",0
1,76,JUNGLE,"[1102, 2003, 3340]",0
2,134,MIDDLE,"[1056, 2003, 2003, 3340]",0
3,523,BOTTOM,"[1055, 2003, 3340]",0
4,350,UTILITY,"[3850, 2003, 2003, 3340]",0
...,...,...,...,...
18264,711,MIDDLE,"[3070, 2003, 2003, 3340]",0
18265,56,JUNGLE,"[1103, 2003, 3364]",0
18266,24,TOP,"[1033, 3340, 2003]",0
18267,221,BOTTOM,"[1055, 2003]",0


In [43]:
champ_item_start_count= champ_item_start.groupby(['version','update_time','championId','lane','stratitem']).count().rename(columns={'win':'game_count'})

In [44]:
champ_item_start_count.sort_values('game_count',ascending=False)

game_count
version       update_time         championId lane    stratitem                     
13.9.506.4846 2023-05-11 11:34:18 81         BOTTOM  [1055, 2003]               128
                                                     [1055, 2003, 3340]         126
                                  145        BOTTOM  [1055, 2003]                88
13.8.505.1214 2023-05-11 11:34:18 81         BOTTOM  [1055, 2003]                84
13.9.506.4846 2023-05-11 11:34:18 222        BOTTOM  [1055, 2003, 3340]          80
...                                                                             ...
13.7.502.2251 2023-05-11 11:34:18 221        BOTTOM  [1055, 2003]                 1
                                  157        TOP     [1055, 2003, 3340]           1
                                             MIDDLE  []                           1
                                  145        MIDDLE  [1083, 2003, 3340]           1
13.9.506.4846 2023-05-11 11:34:18 902        UTILITY [3858, 2003, 3340]           1

[10304 rows x 1 columns]

In [45]:
champ_item_start_count_win = champ_item_start.groupby(['version','update_time','championId','lane','stratitem']).sum()

In [46]:
champ_item_start_count_win.sort_values('win',ascending=False)

win
version       update_time         championId lane    stratitem                    
13.9.506.4846 2023-05-11 11:34:18 81         BOTTOM  [1055, 2003, 3340]         75
                                                     [1055, 2003]               71
                                  222        BOTTOM  [1055, 2003, 3340]         44
13.8.505.1214 2023-05-11 11:34:18 81         BOTTOM  [1055, 2003]               44
13.9.506.4846 2023-05-11 11:34:18 103        MIDDLE  [1056, 2003, 2003, 3340]   39
...                                                                            ...
13.6.499.7758 2023-05-11 11:34:18 16         UTILITY []                          0
13.8.505.1214 2023-05-11 11:34:18 56         JUNGLE  [1103]                      0
                                             MIDDLE  [1036, 2031]                0
                                                     [2033]                      0
13.9.506.4846 2023-05-11 11:34:18 902        UTILITY [3858, 2003, 3340]          0

[10304 rows x 1 columns]

In [47]:
result = champ_item_start_count.join(champ_item_start_count_win)

In [50]:
result.reset_index(inplace=True)

In [52]:
result.sort_values('game_count',ascending=False)

,version,update_time,championId,lane,stratitem,game_count,win
9088,13.9.506.4846,2023-05-11 11:34:18,81,BOTTOM,"[1055, 2003]",128,71
9087,13.9.506.4846,2023-05-11 11:34:18,81,BOTTOM,"[1055, 2003, 3340]",126,75
9600,13.9.506.4846,2023-05-11 11:34:18,145,BOTTOM,"[1055, 2003]",88,37
7167,13.8.505.1214,2023-05-11 11:34:18,81,BOTTOM,"[1055, 2003]",84,44
9794,13.9.506.4846,2023-05-11 11:34:18,222,BOTTOM,"[1055, 2003, 3340]",80,44
...,...,...,...,...,...,...,...
4194,13.7.502.2251,2023-05-11 11:34:18,221,BOTTOM,"[1055, 2003]",1,0
4193,13.7.502.2251,2023-05-11 11:34:18,157,TOP,"[1055, 2003, 3340]",1,0
4192,13.7.502.2251,2023-05-11 11:34:18,157,MIDDLE,[],1,0
4191,13.7.502.2251,2023-05-11 11:34:18,145,MIDDLE,"[1083, 2003, 3340]",1,1


In [53]:
result['win_rate']=round(result.win/result.game_count*100,2)

In [54]:
result['pick_rate']=round(result.game_count/(sum(result.game_count)/10)*100,2)

In [56]:
result.sort_values('game_count',ascending=False)

,version,update_time,championId,lane,stratitem,game_count,win,win_rate,pick_rate
9088,13.9.506.4846,2023-05-11 11:34:18,81,BOTTOM,"[1055, 2003]",128,71,55.47,4.33
9087,13.9.506.4846,2023-05-11 11:34:18,81,BOTTOM,"[1055, 2003, 3340]",126,75,59.52,4.26
9600,13.9.506.4846,2023-05-11 11:34:18,145,BOTTOM,"[1055, 2003]",88,37,42.05,2.98
7167,13.8.505.1214,2023-05-11 11:34:18,81,BOTTOM,"[1055, 2003]",84,44,52.38,2.84
9794,13.9.506.4846,2023-05-11 11:34:18,222,BOTTOM,"[1055, 2003, 3340]",80,44,55.00,2.71
...,...,...,...,...,...,...,...,...,...
4194,13.7.502.2251,2023-05-11 11:34:18,221,BOTTOM,"[1055, 2003]",1,0,0.00,0.03
4193,13.7.502.2251,2023-05-11 11:34:18,157,TOP,"[1055, 2003, 3340]",1,0,0.00,0.03
4192,13.7.502.2251,2023-05-11 11:34:18,157,MIDDLE,[],1,0,0.00,0.03
4191,13.7.502.2251,2023-05-11 11:34:18,145,MIDDLE,"[1083, 2003, 3340]",1,1,100.00,0.03


In [59]:
result = result[['version','update_time','championId','lane','stratitem','pick_rate','game_count','win_rate']]

TypeError: list indices must be integers or slices, not list

In [58]:
result

[['version',
  'update_time',
  'championId',
  'lane',
  'stratitem',
  'pick_rate',
  'game_count',
  'win_rate']]